In [1]:
%%writefile ../src/CusOrder.sol
pragma solidity ^0.6.0;

contract Customer {
    uint id;
    string name;
    string ph;
    string home;
    
    mapping(address => uint256) public balanceOf;
    mapping(string => address) playersAddr;
    
    function addCustomer(uint _id, string memory _name, string memory _ph, string memory _home)  public payable {
        id=_id;
        name=_name;
        ph=_ph;
        home=_home;
    }
    
    function getHomeAddress() public view returns(string memory) {
        return home;
    }
    
    function getId() view public returns(uint) {
        return id;
    }
}


contract Order {
    
    mapping(uint=>Customer) customerById;
    
    struct OrderStruct{
        uint id;
        string p;
        uint n;
        uint amount;
        
        uint time;
        string status;
        string shipping;
    }
    uint orderLen = 0;
    uint totalAmount = 0;
    
    mapping(address=>OrderStruct) orderMap;
    mapping(uint=>address) addressById;
    mapping(address=>string) homeByAddr;
    
    address payable owner;
    constructor() public {
        owner = msg.sender;
    }
    
    
    function placeOrder(uint _id, string memory _p, uint _n, uint _amount) public payable{
        
        string memory  home = getHomeAddress();
        OrderStruct memory o=OrderStruct(_id, _p, _n, _amount, now, "Orderd", home);
        
        orderMap[tx.origin] = o;
        addressById[_id] = tx.origin;
        orderLen += 1;
        totalAmount += _amount;
    }
    
    function addCustomer(uint _id, string memory _name, string memory _ph, string memory _home) public{
        Customer cus = new Customer();
        cus.addCustomer(_id, _name, _ph, _home);
        
        customerById[_id]=cus;
        homeByAddr[tx.origin] = _home;
    }
    

    function getHomeAddress() public view returns(string memory){
        return homeByAddr[tx.origin];
    }
    
    function getStatus() public view returns(string memory){
        return orderMap[tx.origin].status;
    }
    
    function updateStatus(uint _id, string memory _s) public isOwner {
        orderMap[addressById[_id]].status = _s;
    }
    
    function getOrderItem() public view returns(uint, string memory, string memory, string memory){
        return (orderMap[tx.origin].id, orderMap[tx.origin].p, orderMap[tx.origin].status, orderMap[tx.origin].shipping);
    }
    
    function getOrderById(uint _id)  public view returns(uint, string memory, string memory, string memory){
        OrderStruct memory o = orderMap[addressById[_id]];
        return (o.id, o.p, o.status, o.shipping);
    }
    
    function getNOrder() public view returns(uint){
        return orderLen;
    }
    
    function getTotalOrderAmount() public view returns(uint){
        return totalAmount;
    }
    
    function queryBalance()  view public returns(uint){
        return address(this).balance;
    }
    
    modifier isOwner{
        require(msg.sender == owner);
        _;
    }
        
    // }
}

Writing ../src/CusOrder.sol


In [12]:
!solc ../src/CusOrder.sol --combined-json abi > ..\src\CusOrderABI.json

In [13]:
!solc ../src/CusOrder.sol --combined-json bin > ..\src\CusOrderBIN.json

In [15]:
!type ..\src\CusOrderABI.json

{"contracts":{"../src/CusOrder.sol:Customer":{"abi":"[{\"inputs\":[{\"internalType\":\"uint256\",\"name\":\"_id\",\"type\":\"uint256\"},{\"internalType\":\"string\",\"name\":\"_name\",\"type\":\"string\"},{\"internalType\":\"string\",\"name\":\"_ph\",\"type\":\"string\"},{\"internalType\":\"string\",\"name\":\"_home\",\"type\":\"string\"}],\"name\":\"addCustomer\",\"outputs\":[],\"stateMutability\":\"payable\",\"type\":\"function\"},{\"inputs\":[{\"internalType\":\"address\",\"name\":\"\",\"type\":\"address\"}],\"name\":\"balanceOf\",\"outputs\":[{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"}],\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"getHomeAddress\",\"outputs\":[{\"internalType\":\"string\",\"name\":\"\",\"type\":\"string\"}],\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"getId\",\"outputs\":[{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"}],\"stateMutability\":\"view\",\"type\":\"f

## 배포

In [53]:
%%writefile ../src/CusOrderDeploy.js
var Web3=require('web3');
var _abiJson = require('./CusOrderABI.json');
var _binJson = require('./CusOrderBIN.json');

var web3;
web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));


contractName=Object.keys(_abiJson.contracts); 
_abiArray=JSON.parse(_abiJson.contracts[contractName[1]].abi);
_bin=_binJson.contracts[contractName[1]].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();

    //gas check
    new web3.eth.Contract(_abiArray).deploy({data: _bin}).estimateGas(function(err, gas) {
        if(!err) console.log(">> gas: "+ gas);
    });
    
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1662136, gasPrice: '1000000000'}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting ../src/CusOrderDeploy.js


In [54]:
!node ../src/CusOrderDeploy.js

>> gas: 1662136
hash: 0x93c0b79d4db7f44d8eb37026c03dde7567fd394d1234bfdf952a3fc124684b14
---> The contract deployed to: 0x7F1a224C26BA3856442300Edd79D8236bAB6dd10


## 사용

# 3) 계정, 잔고, gas 출력

In [76]:
%%writefile ../src/CusOrderUse.js
var Web3=require('web3');
var _abiJson = require('./CusOrderABI.json');
var _binJson = require('./CusOrderBIN.json');


var web3;
var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://localhost:8345"));
contractName=Object.keys(_abiJson.contracts); 
_bin=_binJson.contracts[contractName[1]].bin;
_abiArray=JSON.parse(_abiJson.contracts[contractName[1]].abi);

var _order = new web3.eth.Contract(_abiArray, '0x7F1a224C26BA3856442300Edd79D8236bAB6dd10');

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    const balance0 = await web3.eth.getBalance(accounts[0]);
    const balance1 = await web3.eth.getBalance(accounts[1]);
    const balance2 = await web3.eth.getBalance(accounts[2]);
    
    console.log(`account0: ${accounts[0]}, balance: ${balance0}`)
    console.log(`account1: ${accounts[0]}, balance: ${balance1}`)
    console.log(`account2: ${accounts[0]}, balance: ${balance2}\n`)
    
    new web3.eth.Contract(_abiArray).deploy({data: _bin}).estimateGas(function(err, gas) {
        if(!err) console.log("gas: "+ gas);
    });
    
    // wait and exit
    setTimeout(function() {
      process.exit(1);
    }, 1000);
}

doIt()

Overwriting ../src/CusOrderUse.js


In [77]:
!node ../src/CusOrderUse.js

account0: 0x1CEc51b65ef4A8b0124581587FC9F846FC2DAF83, balance: 100000000000000000000
account1: 0x1CEc51b65ef4A8b0124581587FC9F846FC2DAF83, balance: 100000000000000000000
account2: 0x1CEc51b65ef4A8b0124581587FC9F846FC2DAF83, balance: 100000000000000000000

gas: 1662136


# 4) 고객정보 3건 입력

In [84]:
%%writefile ../src/CusOrderUse.js
var Web3=require('web3');
var _abiJson = require('./CusOrderABI.json');
var _binJson = require('./CusOrderBIN.json');


var web3;
var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://localhost:8345"));
contractName=Object.keys(_abiJson.contracts); 
_bin=_binJson.contracts[contractName[1]].bin;
_abiArray=JSON.parse(_abiJson.contracts[contractName[1]].abi);

var _order = new web3.eth.Contract(_abiArray, '0x7F1a224C26BA3856442300Edd79D8236bAB6dd10');

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    
    
    await _order.methods.addCustomer(111, "kim", "010-2017-1111", "111 hongji-dong jongro-gu seoul")
    .call();
    
    var home = await _order.methods.getHomeAddress().call()
    console.log(home)
    
    // wait and exit
    setTimeout(function() {
      process.exit(1);
    }, 1000);
}

doIt()

Overwriting ../src/CusOrderUse.js


In [85]:
!node ../src/CusOrderUse.js

(node:16020) UnhandledPromiseRejectionWarning: Error: Returned values aren't valid, did it run Out of Gas? You might also see this error if you are not using the correct ABI for the contract you are retrieving data from, requesting data from a block number that does not exist, or querying a node which is not fully synced.
    at ABICoder.decodeParameters (C:\Users\yoonho\Code\201910812\node_modules\web3-eth-abi\src\index.js:239:15)
    at Contract._decodeMethodReturn (C:\Users\yoonho\Code\201910812\node_modules\web3-eth-contract\src\index.js:557:22)
    at Method.outputFormatter (C:\Users\yoonho\Code\201910812\node_modules\web3-eth-contract\src\index.js:910:46)
    at Method.formatOutput (C:\Users\yoonho\Code\201910812\node_modules\web3-core-method\src\index.js:167:54)
    at sendTxCallback (C:\Users\yoonho\Code\201910812\node_modules\web3-core-method\src\index.js:596:33)
    at Object.<anonymous> (C:\Users\yoonho\Code\201910812\node_modules\web3-core-requestmanager\src\index.js:147:9)